In [ ]:
import os
import json
import numpy as np
import torch
from collections import defaultdict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, MarianMTModel, MarianTokenizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_precision_recall_accuracy_score
import logging
import optuna
from tqdm import tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 20.1 MB/s eta 0:00:00


In [ ]:
# Thiết lập logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
def map_emoji_to_spanish(emoji=None):
    emoji_map = {
        "🔝": "arriba",
        "👎": "no me gusta",
        "😳": "sorprendido",
        "4️⃣": "cuatro",
        "🖐🏼": "mano abierta",
        "💎": "diamante",
        "🤣": "riendo fuerte",
        "🤞🏻": "dedos cruzados",
        "🍺": "cerveza",
        "❣": "corazón exclamación",
        "🤡": "payaso",
        "🎅🏻": "Papá Noel",
        "⬆": "subir",
        "💸": "dinero volando",
        "🤤": "babeando",
        "❌": "cruz",
        "🙌🏻": "manos arriba",
        "🤩": "asombrado",
        "🇵🇪": "Perú",
        "🤠": "vaquero",
        "🟣": "círculo morado",
        "🖐🏽": "mano abierta",
        "🙃": "cara invertida",
        "🐸": "rana",
        "👆🏼": "señalando arriba",
        "🈚": "gratis",
        "🌐": "mundo",
        "🎁": "regalo",
        "🎉": "celebración",
        "😵‍💫": "mareado",
        "🌝": "luna llena",
        "🙋‍♂": "hombre levantando mano",
        "3️⃣": "tres",
        "🔮": "bola de cristal",
        "😰": "nervioso",
        "😨": "miedo",
        "❓": "pregunta",
        "☝🏻": "dedo arriba",
        "🥲": "lágrimas de alegría",
        "✊🏼": "puño levantado",
        "✊": "puño",
        "🧘🏻‍♂": "meditación",
        "🧐": "curioso",
        "👏🏾": "aplausos",
        "🐳": "ballena",
        "💪🏼": "fuerza",
        "✅": "aprobado",
        "🤦🏼‍♂": "vergüenza",
        "😍": "enamorado",
        "👻": "fantasma",
        "😂": "riendo",
        "💪🏻": "fuerte",
        "🫤": "decepción",
        "⚽": "fútbol",
        "🥚": "huevo",
        "🙏": "rezando",
        "🤙": "llámame",
        "🙄": "aburrido",
        "😲": "asombro",
        "♥": "corazón",
        "🍎": "manzana",
        "🐻": "oso",
        "🤪": "loco",
        "👆🏽": "señalando arriba",
        "🎢": "montaña rusa",
        "🙌": "celebrando",
        "🌘": "luna menguante",
        "🫡": "saludo",
        "🙋🏻‍♀": "mujer levantando mano",
        "🤦‍♂": "error",
        "🌊": "ola",
        "😉": "guiño",
        "🥶": "frío",
        "💋": "beso",
        "🇺🇦": "Ucrania",
        "😶‍🌫": "confundido",
        "🌬": "viento",
        "💩": "mierda",
        "👌🏼": "perfecto",
        "🙆‍♂": "hombre OK",
        "💪🏽": "fuerza",
        "😱": "gritando",
        "1️⃣": "uno",
        "🤘": "rock",
        "👉": "señalando derecha",
        "🙂": "sonriendo",
        "👁": "ojo",
        "👀": "ojos",
        "🔥": "fuego",
        "⏺": "grabar",
        "😅": "sudando",
        "❗": "exclamación",
        "😕": "confuso",
        "🥒": "pepino",
        "🎂": "torta",
        "😥": "aliviado",
        "✌🏽": "victoria",
        "🎾": "tenis",
        "💚": "corazón verde",
        "💔": "corazón roto",
        "👍": "bien",
        "🐶": "perro",
        "✔": "verificado",
        "✌🏻": "paz",
        "💪": "músculo",
        "🎈": "globo",
        "🤑": "dinero en la cara",
        "😾": "gato enfadado",
        "💵": "billete",
        "👋🏻": "saludando",
        "👈🏻": "señalando izquierda",
        "💰": "bolsa de dinero",
        "🎼": "música",
        "🐮": "vaca",
        "🇦🇷": "Argentina",
        "🤷🏼‍♀": "mujer encogiéndose",
        "💃": "bailando",
        "🤮": "vomitando",
        "🇷🇺": "Rusia",
        "😎": "genial",
        "🥳": "fiesta",
        "⚰": "ataúd",
        "💯": "cien puntos",
        "📈": "gráfico subiendo",
        "😭": "llorando",
        "😪": "somnoliento",
        "🤞🏼": "suerte",
        "🤦🏽‍♂": "hombre avergonzado",
        "▶": "reproducir",
        "⛔": "prohibido",
        "🎶": "notas musicales",
        "🙊": "mono callado",
        "🌚": "luna nueva",
        "👏": "aplaudiendo",
        "🙏🏽": "rezando",
        "😄": "feliz",
        "🤦🏻‍♂": "error hombre",
        "🇨🇳": "China",
        "👌🏻": "OK",
        "🤙🏻": "llámame",
        "🇳🇬": "Nigeria",
        "😃": "alegre",
        "ℹ️": "información",
        "🗣": "hablando",
        "🙌🏼": "manos levantadas",
        "🤞": "cruzando dedos",
        "😜": "broma",
        "🎵": "nota musical",
        "🤟": "te amo",
        "✈": "avión",
        "👌🏽": "perfecto",
        "🤦🏽": "vergüenza",
        "👍🏾": "bien",
        "🔹": "diamante azul",
        "😝": "lengua fuera",
        "💶": "euro",
        "🤓": "nerd",
        "😶": "sin expresión",
        "🐁": "ratón",
        "🐗": "jabalí",
        "🤦🏻‍♀": "mujer avergonzada",
        "🍏": "manzana verde",
        "🟢": "círculo verde",
        "🙌🏽": "celebración",
        "🇪🇸": "España",
        "✨": "brillo",
        "🤷🏻‍♂": "hombre encogiéndose",
        "🚨": "alarma",
        "🥰": "amor",
        "☺": "sonrisa",
        "🤷‍♂": "duda",
        "🤯": "cabeza explotando",
        "🥺": "suplicando",
        "🐟": "pez",
        "🇮🇳": "India",
        "😐": "neutral",
        "😁": "sonriendo amplio",
        "🙋🏻‍♂": "levantando mano",
        "😓": "sudor",
        "🕺": "bailando",
        "😯": "sorprendido",
        "👉🏻": "señalando derecha",
        "💥": "explosión",
        "😢": "llorando",
        "🦖": "T-Rex",
        "⚡": "rayo",
        "😴": "durmiendo",
        "🫣": "espiando",
        "😻": "gato enamorado",
        "🥵": "caliente",
        "👍🏻": "pulgar arriba",
        "🇧🇾": "Bielorrusia",
        "🤷🏽‍♀": "mujer dudando",
        "😋": "saboreando",
        "🚫": "prohibido",
        "👅": "lengua",
        "😆": "riendo mucho",
        "😊": "sonriendo feliz",
        "😇": "ángel",
        "😠": "enojado",
        "🌎": "Américas",
        "⬇": "bajar",
        "😞": "triste",
        "🔵": "círculo azul",
        "📨": "correo",
        "👆": "arriba",
        "😘": "besando",
        "🌖": "luna gibosa",
        "❤": "corazón rojo",
        "☝": "dedo arriba",
        "✌": "victoria",
        "🍻": "brindis",
        "🤝": "apretón de manos",
        "👋": "saludo",
        "💲": "dólar",
        "👍🏼": "bien",
        "🚶🏻‍♂": "hombre caminando",
        "🤔": "pensando",
        "😹": "gato riendo",
        "🫵": "señalando",
        "🤭": "riendo callado",
        "🪂": "paracaídas",
        "😈": "diablo",
        "🔰": "principiante",
        "🫀": "corazón",
        "😒": "molesto",
        "🤷": "no sé",
        "😀": "felicidad",
        "🍀": "trébol",
        "🔪": "cuchillo",
        "😮": "boca abierta",
        "💬": "hablar",
        "✋": "mano levantada",
        "😌": "alivio",
        "💦": "sudor",
        "🤷🏼‍♂": "duda",
        "☹": "tristeza",
        "🤨": "sospecha",
        "🤙🏽": "llámame",
        "🔻": "triángulo abajo",
        "🛍": "compras",
        "🤧": "estornudo",
        "💫": "mareo",
        "👼": "ángel",
        "🤌": "pellizco",
        "💨": "rápido",
        "😛": "lengua fuera",
        "🎄": "árbol de Navidad",
        "🥹": "lágrimas contenidas",
        "☀": "sol",
        "🌕": "luna llena",
        "🇺🇸": "Estados Unidos",
        "👏🏼": "aplausos",
        "‼": "doble exclamación",
        "🚀": "cohete",
        "😡": "furioso",
        "😬": "nervios",
        "🔴": "círculo rojo",
        "🙏🏻": "orando",
        "🙈": "mono tapándose",
        "🦥": "perezoso",
        "🌙": "luna creciente",
        "👈": "señalando izquierda",
        "🐷": "cerdo",
        "🥸": "disfrazado",
        "😏": "sonrisa pícara",
        "😚": "beso cerrado",
        "⚓": "ancla",
        "👌": "OK",
        "🤟🏻": "te amo",
        "🌌": "vía láctea",
        "⚠": "advertencia",
        "🥱": "bostezando",
        "🐬": "delfín",
        "📊": "gráfico",
        "🐀": "rata",
        "🤗": "abrazo",
        "😔": "pensativo",
        "👏🏻": "aplaudiendo",
        "🇧🇬": "Bulgaria",
        "🥴": "mareado"
    }
    if emoji is None:
        return emoji_map  
    return emoji_map.get(emoji, emoji)  

def replace_emojis_in_text(text):
    result = text
    for emoji, spanish_text in map_emoji_to_spanish().items():
        result = result.replace(emoji, f" {spanish_text} ")
    return result.strip()

def preprocess_data(data_dir):
    subjects = defaultdict(list)
    with tqdm(total=len(os.listdir(data_dir)), desc="Loading JSON Files") as pbar:
        for filename in os.listdir(data_dir):
            if filename.endswith(".json"):
                with open(os.path.join(data_dir, filename), "r") as f:
                    messages = json.load(f)
                    nick = filename.split(".")[0]
                    subjects[nick] = [
                        replace_emojis_in_text(str(msg["message"]) if msg["message"] is not None else "")
                        for msg in messages
                    ]
                pbar.update(1)
    return subjects


In [ ]:
def augment_data(subjects, labels, minority_class=3, save_path="/content/drive/MyDrive/data/task2/train/augmented_data.json"):
    if os.path.exists(save_path):
        with open(save_path, "r") as f:
            augmented_data = json.load(f)
        final_subjects = defaultdict(list)
        final_labels = {}
        for nick, data in augmented_data.items():
            final_subjects[nick] = data["messages"]
            final_labels[nick] = data["label"]
        return final_subjects, final_labels

    es_en_tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-es-en")
    es_en_model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-es-en")
    en_fr_tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")
    en_fr_model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-fr")
    fr_es_tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-fr-es")
    fr_es_model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-fr-es")

    augmented_subjects = defaultdict(list)
    augmented_labels = []
    total_messages_to_translate = sum(len(messages) for nick, messages in subjects.items() if labels[nick] == minority_class)

    with tqdm(total=total_messages_to_translate, desc="Back Translation") as pbar:
        for nick, messages in subjects.items():
            label = labels[nick]
            augmented_subjects[nick] = messages
            augmented_labels.append(label)
            if label == minority_class:
                augmented_messages = []
                for msg in messages:
                    try:
                        es_en_inputs = es_en_tokenizer(msg, return_tensors="pt", padding=True, truncation=True, max_length=512)
                        es_en_outputs = es_en_model.generate(**es_en_inputs)
                        en_text = es_en_tokenizer.decode(es_en_outputs[0], skip_special_tokens=True)
                        en_fr_inputs = en_fr_tokenizer(en_text, return_tensors="pt", padding=True, truncation=True, max_length=512)
                        en_fr_outputs = en_fr_model.generate(**en_fr_inputs)
                        fr_text = en_fr_tokenizer.decode(en_fr_outputs[0], skip_special_tokens=True)
                        fr_es_inputs = fr_es_tokenizer(fr_text, return_tensors="pt", padding=True, truncation=True, max_length=512)
                        fr_es_outputs = fr_es_model.generate(**fr_es_inputs)
                        back_es_text = fr_es_tokenizer.decode(fr_es_outputs[0], skip_special_tokens=True)
                        augmented_messages.append(back_es_text)
                    except Exception as e:
                        augmented_messages.append(msg)
                    pbar.update(1)
                new_nick = f"{nick}_augment"
                augmented_subjects[new_nick] = augmented_messages
                augmented_labels.append(label)
            else:
                pbar.update(len(messages))

    final_subjects = augmented_subjects
    final_labels = dict(zip(final_subjects.keys(), augmented_labels))
    augmented_data = {nick: {"messages": messages, "label": final_labels[nick]} for nick, messages in final_subjects.items()}
    with open(save_path, "w") as f:
        json.dump(augmented_data, f, ensure_ascii=False, indent=4)
    return final_subjects, final_labels

In [ ]:
class AddictionDataset(torch.utils.data.Dataset):
    def __init__(self, subjects, labels, tokenizer, max_length=256):
        self.subjects = subjects
        self.labels = [labels[nick] for nick in subjects.keys()]
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.texts = [' '.join(messages) for messages in subjects.values()]

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        "f1_macro": f1_score(labels, predictions, average='macro'),
        "f1_weighted": f1_score(labels, predictions, average='weighted'),
        "precision": precision_score(labels, predictions, average='macro'),
        "recall": recall_score(labels, predictions, average='macro'),
        "accuracy": accuracy_score(labels, predictions)
    }

In [ ]:
def objective(trial):
    print(f"Starting Trial {trial.number}")

    learning_rate = trial.suggest_float("learning_rate", 3e-4, 5e-4, log=True)
    batch_size = trial.suggest_categorical("batch_size", [2, 4])
    num_epochs = trial.suggest_int("num_epochs", 4, 10)
    weight_decay = trial.suggest_float("weight_decay", 0.05, 0.1)
    print(f"Trial {trial.number} parameters: lr={learning_rate}, batch_size={batch_size}, "
          f"epochs={num_epochs}, weight_decay={weight_decay}")

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")
    if device == 'cuda':
        print(f"GPU memory allocated: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")

    logger.info("Loading roberta-base-bne model...")
    print("Loading tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained("PlanTL-GOB-ES/roberta-base-bne")
    print("Tokenizer loaded successfully")

    print("Loading data...")
    subjects, labels = load_data('/content/drive/MyDrive/data/task2/train/subjects')
    print(f"Loaded {len(subjects)} subjects and {len(labels)} labels")
    subjects, labels = augment_data(subjects, labels)
    print(f"After augmentation: {len(subjects)} subjects and {len(labels)} labels")

    print("Setting up Stratified K-Fold Cross-Validation...")
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    fold_scores = []

    for fold, (train_idx, val_idx) in enumerate(skf.split(list(subjects.keys()), list(labels.values()))):
        print(f"\n=== Trial {trial.number} - Starting Fold {fold + 1}/{skf.n_splits} ===")

        train_subjects = {list(subjects.keys())[i]: subjects[list(subjects.keys())[i]] for i in train_idx}
        val_subjects = {list(subjects.keys())[i]: subjects[list(subjects.keys())[i]] for i in val_idx}
        train_labels = {list(subjects.keys())[i]: labels[list(subjects.keys())[i]] for i in train_idx}
        val_labels = {list(subjects.keys())[i]: labels[list(subjects.keys())[i]] for i in val_idx}
        print(f"Trial {trial.number} - Fold {fold + 1}: Training samples: {len(train_subjects)}, "
              f"Validation samples: {len(val_subjects)}")

        print(f"Trial {trial.number} - Fold {fold + 1}: Creating datasets...")
        train_dataset = AddictionDataset(train_subjects, train_labels, tokenizer)
        val_dataset = AddictionDataset(val_subjects, val_labels, tokenizer)
        print(f"Trial {trial.number} - Fold {fold + 1}: Train dataset size: {len(train_dataset)}")
        print(f"Trial {trial.number} - Fold {fold + 1}: Validation dataset size: {len(val_dataset)}")


        print(f"Trial {trial.number} - Fold {fold + 1}: Loading model...")
        model = AutoModelForSequenceClassification.from_pretrained(
            "PlanTL-GOB-ES/roberta-base-bne",
            num_labels=4,
            ignore_mismatched_sizes=True,
            hidden_dropout_prob=0.1,
            attention_probs_dropout_prob=0.1
        )
        print(f"Trial {trial.number} - Fold {fold + 1}: Model loaded successfully")

        class_weights = torch.tensor([1.0, 1.0, 1.0, 2]).to(device)
        print(f"Trial {trial.number} - Fold {fold + 1}: Class weights: {class_weights}")

        print(f"Trial {trial.number} - Fold {fold + 1}: Setting up TrainingArguments...")
        training_args = TrainingArguments(
            output_dir=f'./results/trial_{trial.number}_fold_{fold + 1}',
            num_train_epochs=num_epochs,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            warmup_steps=500,
            weight_decay=weight_decay,
            logging_dir=f'./logs/trial_{trial.number}_fold_{fold + 1}',
            logging_steps=10,
            eval_strategy="epoch",
            save_strategy="epoch",
            load_best_model_at_end=True,
            metric_for_best_model="f1_macro",
            greater_is_better=True,
            fp16=True if torch.cuda.is_available() else False,
        )
        print(f"Trial {trial.number} - Fold {fold + 1}: TrainingArguments configured: "
              f"epochs={num_epochs}, batch_size={batch_size}")

        def compute_metrics(eval_pred):
            logits, labels = eval_pred
            predictions = np.argmax(logits, axis=-1)
            metrics = {
                "f1_macro": f1_score(labels, predictions, average='macro'),
                "f1_weighted": f1_score(labels, predictions, average='weighted'),
                "precision": precision_score(labels, predictions, average='macro'),
                "recall": recall_score(labels, predictions, average='macro'),
                "accuracy": accuracy_score(labels, predictions)
            }
            print(f"Trial {trial.number} - Fold {fold + 1} Evaluation Metrics: {metrics}")
            return metrics

        print(f"Trial {trial.number} - Fold {fold + 1}: Initializing WeightedTrainer...")
        class WeightedTrainer(Trainer):
            def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
                labels = inputs.pop("labels")
                outputs = model(**inputs)
                logits = outputs.logits
                loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights)
                loss = loss_fct(logits, labels)
                return (loss, outputs) if return_outputs else loss

        trainer = WeightedTrainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            compute_metrics=compute_metrics
        )
        print(f"Trial {trial.number} - Fold {fold + 1}: WeightedTrainer initialized")

        logger.info(f"Trial {trial.number} - Fold {fold + 1}: Training model...")
        print(f"Trial {trial.number} - Fold {fold + 1}: Starting training...")
        try:
            trainer.train()
            print(f"Trial {trial.number} - Fold {fold + 1}: Training completed")
        except Exception as e:
            print(f"Trial {trial.number} - Fold {fold + 1}: Error during training: {str(e)}")
            raise e

        logger.info(f"Trial {trial.number} - Fold {fold + 1}: Evaluating model...")
        print(f"Trial {trial.number} - Fold {fold + 1}: Evaluating model...")
        try:
            eval_results = trainer.evaluate()
            print(f"Trial {trial.number} - Fold {fold + 1} Results: {eval_results}")
            fold_scores.append(eval_results['eval_f1_macro'])
        except Exception as e:
            print(f"Trial {trial.number} - Fold {fold + 1}: Error during evaluation: {str(e)}")
            raise e
    avg_f1 = np.mean(fold_scores)
    print(f"Trial {trial.number} - Average F1 Macro across folds: {avg_f1:.4f}")
    return avg_f1

In [ ]:
def train_and_save_model(data_dir, output_dir='./best_model'):
    logger.info("Optimizing hyperparameters with Optuna...")
    study = optuna.create_study(direction="maximize")
    with tqdm(total=20, desc="Optuna Trials") as pbar:
        def callback(study, trial):
            pbar.update(1)
        study.optimize(objective, n_trials=20, callbacks=[callback])

    best_params = study.best_params
    logger.info(f"Best hyperparameters: {best_params}")

    subjects, labels = load_data(data_dir)
    subjects, labels = augment_data(subjects, labels)

    logger.info("Loading final roberta-base-bne model...")
    tokenizer = AutoTokenizer.from_pretrained("PlanTL-GOB-ES/roberta-base-bne")

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    fold_scores = []
    best_f1 = 0.0
    best_model_dir = f"{output_dir}_best"

    for fold, (train_idx, val_idx) in enumerate(skf.split(list(subjects.keys()), list(labels.values()))):
        print(f"\n=== Starting Fold {fold + 1}/{skf.n_splits} ===")

        train_subjects = {list(subjects.keys())[i]: subjects[list(subjects.keys())[i]] for i in train_idx}
        val_subjects = {list(subjects.keys())[i]: subjects[list(subjects.keys())[i]] for i in val_idx}
        train_labels = {list(subjects.keys())[i]: labels[list(subjects.keys())[i]] for i in train_idx}
        val_labels = {list(subjects.keys())[i]: labels[list(subjects.keys())[i]] for i in val_idx}

        print(f"Fold {fold + 1}: Training samples: {len(train_subjects)}, Validation samples: {len(val_subjects)}")

        train_dataset = AddictionDataset(train_subjects, train_labels, tokenizer)
        val_dataset = AddictionDataset(val_subjects, val_labels, tokenizer)

        model = AutoModelForSequenceClassification.from_pretrained(
            "PlanTL-GOB-ES/roberta-base-bne",
            num_labels=4,
            ignore_mismatched_sizes=True,
            hidden_dropout_prob=0.2,
            attention_probs_dropout_prob=0.2
        )

        training_args = TrainingArguments(
            output_dir=f'./results/fold_{fold + 1}',
            num_train_epochs=best_params['num_epochs'],
            per_device_train_batch_size=best_params['batch_size'],
            per_device_eval_batch_size=best_params['batch_size'],
            warmup_steps=500,
            weight_decay=best_params['weight_decay'],
            learning_rate=best_params['learning_rate'],
            logging_dir=f'./logs/fold_{fold + 1}',
            logging_steps=10,
            eval_strategy="epoch",
            save_strategy="epoch",
            load_best_model_at_end=True,
            metric_for_best_model="f1_macro",
            greater_is_better=True,
            gradient_accumulation_steps=2,
            fp16=True if torch.cuda.is_available() else False,
        )

        def compute_metrics(eval_pred):
            logits, labels = eval_pred
            predictions = np.argmax(logits, axis=-1)
            metrics = {
                "f1_macro": f1_score(labels, predictions, average='macro'),
                "f1_weighted": f1_score(labels, predictions, average='weighted'),
                "precision": precision_score(labels, predictions, average='macro'),
                "recall": recall_score(labels, predictions, average='macro'),
                "accuracy": accuracy_score(labels, predictions)
            }
            return metrics

        class_weights = torch.tensor([1.0, 1.0, 1.0, 1.5]).to('cuda' if torch.cuda.is_available() else 'cpu')

        class WeightedTrainer(Trainer):
            def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
                labels = inputs.pop("labels")
                outputs = model(**inputs)
                logits = outputs.logits
                loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights)
                loss = loss_fct(logits, labels)
                return (loss, outputs) if return_outputs else loss

        trainer = WeightedTrainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            compute_metrics=compute_metrics
        )

        print(f"Fold {fold + 1}: Starting training...")
        trainer.train()

        print(f"Fold {fold + 1}: Evaluating model...")
        eval_results = trainer.evaluate()
        fold_scores.append(eval_results)

        print(f"Fold {fold + 1} Results: {eval_results}")

        if eval_results['eval_f1_macro'] > best_f1:
            best_f1 = eval_results['eval_f1_macro']
            print(f"Fold {fold + 1}: New best model found with F1 macro = {best_f1}. Saving model...")
            trainer.save_model(best_model_dir)
            tokenizer.save_pretrained(best_model_dir)

    avg_metrics = {
        "f1_macro": np.mean([score['eval_f1_macro'] for score in fold_scores]),
        "f1_weighted": np.mean([score['eval_f1_weighted'] for score in fold_scores]),
        "precision": np.mean([score['eval_precision'] for score in fold_scores]),
        "recall": np.mean([score['eval_recall'] for score in fold_scores]),
        "accuracy": np.mean([score['eval_accuracy'] for score in fold_scores])
    }

    print("\n=== Cross-Validation Summary ===")
    print(f"Average F1 Macro: {avg_metrics['f1_macro']:.4f}")
    print(f"Average F1 Weighted: {avg_metrics['f1_weighted']:.4f}")
    print(f"Average Precision: {avg_metrics['precision']:.4f}")
    print(f"Average Recall: {avg_metrics['recall']:.4f}")
    print(f"Average Accuracy: {avg_metrics['accuracy']:.4f}")
    print(f"Best model saved to {best_model_dir}")

    logger.info(f"Cross-Validation completed. Average F1 Macro: {avg_metrics['f1_macro']:.4f}")
    logger.info(f"Best model saved to {best_model_dir}")

In [ ]:
if __name__ == "__main__":
    data_dir = '/content/drive/MyDrive/data/task2/train/subjects'
    train_and_save_model(data_dir)

[I 2025-04-20 09:02:48,074] A new study created in memory with name: no-name-50c919f9-f454-49e9-9284-be58b56fb81b
Optuna Trials:   0%|          | 0/20 [00:00<?, ?it/s]

Starting Trial 0
Trial 0 parameters: lr=0.00031216163589308947, batch_size=4, epochs=7, weight_decay=0.060626921945855616
Using device: cuda
GPU memory allocated: 0.00 GB
Loading tokenizer...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/851k [00:00<?, ?B/s]

...

Trial 0 - Fold 4 Evaluation Metrics: {'f1_macro': 0.5485093167701863, 'f1_weighted': 0.5931296066252588, 'precision': 0.5917487684729064, 'recall': 0.6, 'accuracy': 0.6933333333333334}


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Trial 0 - Fold 4 Evaluation Metrics: {'f1_macro': 0.9436767086508469, 'f1_weighted': 0.9455879759413536, 'precision': 0.9636363636363636, 'recall': 0.9308823529411764, 'accuracy': 0.9466666666666667}
Trial 0 - Fold 4 Evaluation Metrics: {'f1_macro': 0.9246562616127834, 'f1_weighted': 0.9346314876749661, 'precision': 0.9159919028340081, 'recall': 0.9457671957671958, 'accuracy': 0.9333333333333333}


[W 2025-04-20 09:17:05,959] Trial 0 failed with parameters: {'learning_rate': 0.00031216163589308947, 'batch_size': 4, 'num_epochs': 7, 'weight_decay': 0.060626921945855616} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "<ipython-input-13-d39d4cd5772d>", line 129, in objective
    trainer.train()
  File "/usr/local/lib/python3.11/dist-packages/transformers/trainer.py", line 2245, in train
    return inner_training_loop(
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transformers/trainer.py", line 2611, in _inner_training_loop
    self.optimizer.step()
  File "/usr/local/lib/python3.11/dist-packages/accelerate/optimizer.py", line 165, in step
    self.scaler.step(self.optimizer, closure)
  File "/usr/local/lib/python3.11/dist-packages/torch/

KeyboardInterrupt: 

In [ ]:
def predict_addiction(json_file_path, model_path='./best_model_best'):
    tokenizer = AutoTokenizer.from_pretrained("PlanTL-GOB-ES/roberta-base-bne")
    model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=4)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)
    model.eval()

    with open(json_file_path, 'r') as f:
        messages = json.load(f)

    processed_messages = [
        replace_emojis_in_text(str(msg["message"]) if msg["message"] is not None else "")
        for msg in messages
    ]
    combined_text = ' '.join(processed_messages)

    encoding = tokenizer(
        combined_text,
        add_special_tokens=True,
        max_length=256,
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=-1).item()

    addiction_types = {0: "betting", 1: "onlinegaming", 2: "trading", 3: "lootboxes"}
    return addiction_types[predicted_class]

In [ ]:
if __name__ == "__main__":
    data_dir = '/content/drive/MyDrive/data/task2/train/subjects'
    train_and_save_model(data_dir)
    json_file_path = 'path_to_your_json_file.json'
    model_path = './results/trial_0_fold_3/checkpoint-380'
    try:
        addiction = predict_addiction(json_file_path, model_path)
        print(f"Loại nghiện: {addiction}")
    except Exception as e:
        print(f"Lỗi: {str(e)}")